# Métodos basados en frecuencias

También llamados estadísticos, estos métodos explotan estadísticas numéricas con la intención de reflejar que tan importante es una palabra dentro de un documento. 
Estos modelos fueron populares durante los 80s y 90s.
En general, tratan de capturar la occurencia de palabras dentro de todo un corpus.

En este notebook veremos algunos análisis básicos que podemos hacer utilizando este concepto:

In [ ]:
import nltk
from nltk import bigrams, ngrams
from nltk.tokenize.destructive import NLTKWordTokenizer
import pandas as pd

Definimos un corpus simple, extraido de Wikipedia para la definición de hielo

In [ ]:
corpus = ["El hielo es agua en estado sólido",
          "El hielo es uno de los cuatro estados naturales del agua",
          "El agua pura se congela a 0 grados",
          "El hielo es el nombre común del agua en estado sólido"]

tokenizer = NLTKWordTokenizer()
tokenized = [ tokenizer.tokenize(sentense) for sentense in corpus]

In [ ]:
vocab = set(' '.join(corpus).split())

## Co-ocurrencias

Construimos la matrix de co-ocurrencia de palabras, donde basicamente tenemos una matriz de n x n, siendo n el tamaño del vocabulario, y asignando como valor a la cantidad de veces que aparece una palabra en el contexto de la otra. En este caso, **el contexto son todas las palabras del documento**

In [ ]:
co_matrix = pd.DataFrame(0, index=vocab, columns=vocab)
for doc in tokenized:
    for word in doc:
        co_matrix.loc[doc, word] += 1

Exploramos los resultados

In [ ]:
co_matrix

## N-Grams

Otra alternativa, es armar la matriz de co-ocurrencia utilizando n-grams, definiendo una ventana de relevancia. En este caso, también tenemos una matriz de n x n, donde n es el tamaño del vocabulario, y donde el valor de cada elemento es la cantidad de veces que aparece cada palabra en cada contexto. **En este caso, el contexto es una ventana de tamaño k, con k <= len(doc)**. En el siguiente caso por ejemplo, utilizamos bigrams, que es un caso de n-grams con k=2

In [ ]:
bi_grams = list(bigrams(' '.join(corpus).split()))
bigram_freq = nltk.FreqDist(bi_grams)

Construimos la matriz iterando sobre todas las frecuencias de n-grams

In [ ]:
co_matrix = pd.DataFrame()
for word_pair, freq in bigram_freq.items():
    co_matrix.loc[word_pair[0], word_pair[1]] = freq

co_matrix.fillna(0, inplace=True)

In [ ]:
co_matrix

### Visualización de la conexión de las palabras
Podemos visualizar estos bigrams con la ayuda de un grafo, donde los nodos son las palabras y los vertices indican capabras que aparecen en conjunto:

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()
for k, v in bigram_freq.items():
    G.add_edge(k[0], k[1], weight=(v * 10))

Podemos graficar ahora nuesto grafo con una rutina como la siguiente:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))
pos = nx.spring_layout(G, k=2)

# Plot networks
nx.draw_networkx(G, pos,
                 font_size=16,
                 width=3,
                 edge_color='grey',
                 node_color='purple',
                 with_labels = False,
                 ax=ax)

# Create offset labels
for key, value in pos.items():
    x, y = value[0]+.135, value[1]+.045
    ax.text(x, y,
            s=key,
            bbox=dict(facecolor='red', alpha=0.25),
            horizontalalignment='center', fontsize=13)
    
plt.show()